In [28]:
# Library Imports
import pandas as pd
import plotly.express as px
import json

In [2]:
# Data Source
df = pd.read_csv('files/PopulationByCounty1841-2022.csv')

In [3]:
# Printing data to test
df.head()

,Statistic Label,CensusYear,County,Sex,UNIT,VALUE
0,Population at Each Census,1841,State,Both sexes,Number,6528799
1,Population at Each Census,1841,State,Male,Number,3222485
2,Population at Each Census,1841,State,Female,Number,3306314
3,Population at Each Census,1841,Carlow,Both sexes,Number,86228
4,Population at Each Census,1841,Carlow,Male,Number,42428


### Note:
Next step is to pivot this df so that each county has one row per year. The 'Male', 'Female' and 'Both sexes' will get their own column. The *UNIT* and *Statistic Label* columns will be dropped. I will be using the Pandas [*pivot_table*](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.pivot_table.html) function to perform this transformation. 

In [9]:
census_pop = df.pivot_table(
    index=["CensusYear","County"], 
    columns="Sex",
    values="VALUE",
    aggfunc="sum"
).reset_index()

census_pop.columns.name=None
census_pop = census_pop.rename_axis(None, axis=1)

# Re-ordering Columns
census_pop = census_pop[["CensusYear", "County", "Female", "Male"]]

In [10]:
census_pop

,CensusYear,County,Female,Male
0,1841,Carlow,43800,42428
1,1841,Cavan,122344,120814
2,1841,Clare,142285,144109
3,1841,Cork,433567,420551
4,1841,Donegal,150627,145821
...,...,...,...,...
697,2022,Tipperary,84256,83639
698,2022,Waterford,64268,63095
699,2022,Westmeath,48500,47721
700,2022,Wexford,83142,80777


I want to plot the above data using [Plotly](https://plotly.com/python/getting-started/#jupyter-notebook-support) and will toy around with the various plot/graph/chart types below. The first few plots will be a national outlook and I will then focus on the regional area under the [Western Development Commision's](https://westerndevelopment.ie/) remit - **Clare**, **Galway**, **Mayo**, **Roscommon**, **Sligo**, **Leitrim** and **Donegal**. As an area that I was born in to, emigrated from and returned to raise a family in, I have a vested interest in the region's success and analysis of the population should form part of any meaningful analysis for this area. 

In [18]:
# Male vs Female population trends
census_pop["Both"] = census_pop["Male"] + census_pop["Female"]
cp_year = census_pop.groupby("CensusYear")[["Male", "Female", "Both"]].sum().reset_index()

fig = px.line(
    cp_year,
    x="CensusYear",
    y=["Male", "Female", "Both"],
    title="Population of Ireland by Sex Over Time",
    labels={"value": "Population", "CensusYear": "Year", "variable": "Sex"}
)
fig.show()

In [ ]:
# Defined County Population Comparison of Male vs Female

fig = px.line(
    census_pop[census_pop["County"] == "Dublin"],
    x="CensusYear",
    y=["Male", "Female"],
    title="Male vs Female Population in Dublin Over Time",
    labels={"value": "Population", "CensusYear": "Year", "variable": "Sex"}
)
fig.show()


In [ ]:
fig = px.line(
    census_pop[census_pop["County"] == "Mayo"],
    x="CensusYear",
    y=["Male", "Female"],
    title="Male vs Female Population in Mayo Over Time",
    labels={"value": "Population", "CensusYear": "Year", "variable": "Sex"}
)
fig.show()

In [ ]:
fig = px.line(
    census_pop[census_pop["County"] == "Roscommon"],
    x="CensusYear",
    y=["Male", "Female"],
    title="Male vs Female Population in Roscommon Over Time",
    labels={"value": "Population", "CensusYear": "Year", "variable": "Sex"}
)
fig.show()

In [25]:
fig = px.bar(
    census_pop[census_pop["CensusYear"] == 2022],
    x="County",
    y=["Male", "Female"],
    barmode="group",
    title="Population by County and Sex (2022)",
    labels={"value": "Population", "variable": "Sex"}
)
fig.show()


In [27]:
census_pop["Male_to_Female_Ratio"] = census_pop["Male"] / census_pop["Female"]

fig = px.line(
    census_pop.groupby("CensusYear")["Male_to_Female_Ratio"].mean().reset_index(),
    x="CensusYear",
    y="Male_to_Female_Ratio",
    title="Average Male-to-Female Ratio Over Time",
)
fig.show()


## Using a Choropleth Map
---
The Plotly libary offers a *choropleth* function that adds mapping to the visualisations. I sourced an Irish GeoJSON file on [GitHub from 'vool'](https://gist.github.com/vool/969e3be0cfac519560755cce0b91e097#file-ireland-geojson). [The Choropleth documentation from Plotly](https://plotly.com/python/choropleth-maps/) was used to achieve this. 

In [ ]:
# Load the GeoJSON data
with open("files/ireland.geojson") as f:
    counties_geo = json.load(f)

# 

fig = px.choropleth(
    census_pop.assign(Both=census_pop["Male"] + census_pop["Female"]),
    geojson=counties_geo,
    locations="County",
    featureidkey="properties.NAME_TAG",
    color="Both",
    hover_name="County",
    animation_frame="CensusYear",
    color_continuous_scale="Viridis",
    title="Population by County Over Time"
)
fig.update_geos(fitbounds="locations", visible=False)
fig.show()
